# Jagdeep Singh - Protect Earth

The goal of this project is to use your Astro 300 python programming skills to answer the 3 questions below.

Your aim is to:

- Create a well commmented python notebook that shows your programming.
- Keep to the class style guide.
- Do not hard code any common physical constants.
- Easy to read and neat output that **clearly** shows the answers to the questions.
- There should be no calculations outside of the `class` definition.

The starting point is the dataset **`PHA.cvs`** that contains data for 10 objects classified as potentially hazardous asteroids.

In [1]:
import numpy as np
import pandas as pd

from astropy.table import QTable
from astropy import units as u
from astropy import constants as const

In [2]:
## Define unit, ton of TNT, as equal to 4.18x10^9 J
ton_TNT = u.def_unit('ton(s) of TNT', (4.18 * (10 ** 9)) * u.J)
    
class SpaceRock(object):
    
    """Container for Potentially Hazardous Asteroids(PHA)"""
    
    def __init__(self, name = None, ab_mag = None, albedo = None, 
                 semi_major= None, ecc = None):
        
        """
        Parameters
        ----------
        name : string
            Name of the potentially hazardous asteroid
        ab_mag : array-like
            Absolute Magnitude of each Asteroid
        albedo : array-like
            Albedo of each Asteroid
        semi_major : array-like
            Semi Major Axis of each Asteroid in AU
        ecc : array-like
            Eccentricity of each Asteroid
        """
        
        self.name = name
        self.ab_mag = ab_mag
        self.albedo = albedo
        self.semi_major = semi_major * u.AU
        self.ecc = ecc
    
    def diameter(self):
        """
        Determine the diameter (in km) of the PHA
        """
        result = ((1329.0 / np.sqrt(self.albedo)) 
                    * (10 ** (-0.2 * self.ab_mag)))
        result = result * u.km
        return result
    
    def speed(self):
        """
        Determine the speed (in m/s) of the PHA
        """
        result = (np.sqrt(const.G * const.M_sun * 
                          ((2/ (1 * u.AU)) - (1/self.semi_major))))
        result = result.to(u.km / u.s)
        return result
    
    def mass(self):
        """
        Determine the mass (in kg) of the PHA
        """
        density = 3000 * (u.kg / (u.m ** 3))
        result = density * (1 / 6) * np.pi * ((self.diameter()) ** 3)
        result = result.to(u.kg)
        return result.decompose()
    
    def encounter_speed(self):
        """
        Determine the encounter speed (in m/s) of the PBA
        """
        result = self.speed() - (30 * (u.km / u.s)).to(u.m / u.s)
        return result.decompose()
    
    def impact_speed(self):
        """
        Determine the impact speed (in m/s) of the PBA
        """
        result = np.sqrt((self.encounter_speed()) ** 2 
                         + (11.2 * (u.km / u.s)) ** 2).to(u.km / u.s)
        return result
    
    def kinetic_energy(self):
        """
        Determine the kinetic energy (in tons of TNT) of the PHA
        """
        
        ## Define unit, million ton(s) of TNT, as equal to 4.18x10^15 J
        ton_TNT = u.def_unit('million ton(s) of TNT', (4.18 * (10 ** 15)) * u.J)
        
        result = (0.5) * self.mass() * (self.impact_speed() ** 2)
        return result.to(ton_TNT)
    
    def potential_energy(self):
        """
        Determine the potential energy of the PHA
        """

        ## Define unit, 1 ton nuclear weaponds, as equal to 4.18x10^9 J
        ton_nuke = u.def_unit('1 ton nuclear weaponds', (4.18 * (10 ** 9)) * u.J)
        
        result = ((6/5) * ((const.G * 
                            (self.mass() ** 2)) / self.diameter()))
        return result.to(ton_nuke)

---

# There should only be calls to the SpaceRock class below this line 
# (and formatting)

---

## 0 - Read the dataset `PHA.cvs` and call `SpaceRock`

In [3]:
## gets in a table of PHAs from file PHA.csv
pha_table = QTable.read('PHA.csv', format='ascii.csv')

my_name = pha_table['Name'] ## name of PHA
my_semi_major = pha_table['a'] ## semi-major of PHA
my_ecc = pha_table['ecc'] ## eccentricity of PHA
my_ab_mag = pha_table['H'] ## absolute magnitude of PHA
my_albedo = pha_table['A'] ## albedo of PHA

"""
Create array of SpaceRock objects
with properties
    Name
    Absolute Magnitude
    Albedo
    Semi-major axis
    Eccentricity

"""
phas = SpaceRock(name=my_name, ab_mag=my_ab_mag, albedo=my_albedo, 
                 semi_major=my_semi_major, ecc=my_ecc)

## 1 - Determine the speed of each of the PHAs at r = 1 AU.


* Make sure you use units.
* Express your answer SI units with 2 digits after the decimal.
* The output should be a series of lines like:
  * `The speed of NAME at 1 AU is VALUE UNIT.` 

In [4]:
"""
Goes through array of SpaceRock objects and 
prints out speed of each PHA in format:
"The speed of NAME at 1 AU is VALUE UNIT."

Units:
meters per second
"""
for idx,value in enumerate(phas.speed()):
    pha_name = phas.name[idx]
    my_string = ("The speed of {0} at 1 AU is {1:.2f}."
                 .format(pha_name, value))
    print(my_string)

The speed of Icarus at 1 AU is 30.87 km / s.
The speed of Geographos at 1 AU is 32.63 km / s.
The speed of Apollo at 1 AU is 34.22 km / s.
The speed of Midas at 1 AU is 35.72 km / s.
The speed of Adonis at 1 AU is 36.05 km / s.
The speed of Phaethon at 1 AU is 32.80 km / s.
The speed of Dionysus at 1 AU is 37.03 km / s.
The speed of Wilson-Harrington at 1 AU is 37.92 km / s.
The speed of Vishnu at 1 AU is 30.62 km / s.
The speed of Toutatis at 1 AU is 37.73 km / s.


## 2 - Determine the kinetic energy each PHA whould have impact the surface of the Earth

- Express your answer in million of tons of TNT with 1 digit after the decimal
- 1 ton TNT $= 4.18 \times 10^9$ J.
* The output should be a series of lines like:
  * `The kinetic energy of NAME hitting the Earth is VALUE UNIT.` 

In [5]:
"""
Goes through array of SpaceRock objects and prints out 
kinetic energy each PHA would have if it impacted the 
surface of the Earth.
"""
for idx,value in enumerate(phas.kinetic_energy()):
    
    pha_name = phas.name[idx]
    my_string = ("The kinetic energy of {0} hitting the Earth is {1:.1f}."
                 .format(pha_name, value))
    print(my_string)

The kinetic energy of Icarus hitting the Earth is 2173.0 million ton(s) of TNT.
The kinetic energy of Geographos hitting the Earth is 134406.7 million ton(s) of TNT.
The kinetic energy of Apollo hitting the Earth is 89862.4 million ton(s) of TNT.
The kinetic energy of Midas hitting the Earth is 423300.9 million ton(s) of TNT.
The kinetic energy of Adonis hitting the Earth is 3001.4 million ton(s) of TNT.
The kinetic energy of Phaethon hitting the Earth is 2799844.2 million ton(s) of TNT.
The kinetic energy of Dionysus hitting the Earth is 174142.4 million ton(s) of TNT.
The kinetic energy of Wilson-Harrington hitting the Earth is 1891116.7 million ton(s) of TNT.
The kinetic energy of Vishnu hitting the Earth is 1349.7 million ton(s) of TNT.
The kinetic energy of Toutatis hitting the Earth is 431749.1 million ton(s) of TNT.


## 3 - Determine how many 1 ton nuclear weapons will be needed to destroy each of the PHAs.

- Assume $\rho$ = 3,000 kg/m$^3$
- Express your answer in the number of 1 ton weapons with 1 digit after the decimal
* The output should be a series of lines like:
  * `It would take VALUE 1 ton nuclear weaponds to destroy NAME.` 

In [6]:
for idx,value in enumerate(phas.potential_energy()):
    pha_name = phas.name[idx]
    my_string = ("It would take {0:.1f} to destroy {1}."
                .format(value, pha_name))
    print(my_string)

It would take 0.9 1 ton nuclear weaponds to destroy Icarus.
It would take 787.0 1 ton nuclear weaponds to destroy Geographos.
It would take 352.7 1 ton nuclear weaponds to destroy Apollo.
It would take 3957.4 1 ton nuclear weaponds to destroy Midas.
It would take 1.0 1 ton nuclear weaponds to destroy Adonis.
It would take 122681.6 1 ton nuclear weaponds to destroy Phaethon.
It would take 762.0 1 ton nuclear weaponds to destroy Dionysus.
It would take 35878.6 1 ton nuclear weaponds to destroy Wilson-Harrington.
It would take 0.4 1 ton nuclear weaponds to destroy Vishnu.
It would take 3143.5 1 ton nuclear weaponds to destroy Toutatis.


## Due Tue Oct 31 - 5pm
- `Make sure to change the filename to your name!`
- `Make sure to change the Title to your name!`
- `File -> Download as -> HTML (.html)`
- `upload your .html and .ipynb file to the class Canvas page`  

---

## Some Orbital Mechanics

Kepler's first law says: *The orbit of every planet is an ellipse with the sun at one focus*. The Semimajor axis **a** and the eccentricity **ecc** parametrize the size and shape of the ellipse. The units of **a** in our dataset are Astronomical Units (AU), the average distance between the Sun and the Earth.

![Orbit Diagram](images/Orbit.jpg)

For a closed elliptical orbit (orbits gravitationally bound to the Sun), $ecc = \sqrt{1 - {b^2}/{a^2}}$, where **a** and **b** are the semimajor and semiminor axes. As you can see from the equation, when **a** = **b**, **ecc** = 0 (a circle), and when **a** $>>$ **b**, **ecc** approaches 1. When **ecc** = 1, the orbit is a parabolic orbit (just bound). When **ecc** $>$ 1 the orbit is a hyperbolic orbit (unbound).

---

The speed of an object on an elliptical orbit around the Sun at a distance **r** from the Sun is:

$$ \large
v\ =\ \sqrt{GM_{\odot}\ \left(\frac{2}{r} - \frac{1}{a}\right)}
$$

---

## Encountering the Earth

The encounter speed of an asteroid meeting the Earth at 1 AU is (assuimg aligned prograde orbits):

$$ \large
V_{\textrm{encounter}}\ =\ V_{\textrm{asteroid at 1AU}}\ -\ V_{\textrm{Earth}}
$$

Where $V_{\textrm{Earth}}\ =\ 30\ \textrm{km/s}$

## Hitting the Earth

The impact speed of an asteroid hitting the Earth is:

$$ \large
V_{\textrm{impact}}\ =\ \sqrt{V_{\textrm{encounter}}^2 + V_{\textrm{escape}}^2}
$$

Where $V_{\textrm{escape}}\ = 11.2\ km/s$

---

## Blowing up an asteroid

The self gravitational potential energy of a uniform sphere of mass (m) and diameter (d) is:

$$ \large
PE \ = \ \frac{6}{5} \cdot \frac{Gm^2}{d}
$$  

This is the amount of energy you need to give the sphere to move all of its components pieces infinitely far away (i.e. to destroy it!).

Remember that the mass and diameter of the asteroid is derived from its absolute magnitude **H** and albedo **A**.